* The following code is modified based on the original torchcv project. 
* We are going to use PASACAL VOC12 as dataset.
* You could donwload VOC2012 
  train/validation: http://host.robots.ox.ac.uk/pascal/VOC/voc2012/#devkit
* test data: https://pjreddie.com/projects/pascal-voc-dataset-mirror/


In [2]:
import os
import cv2
import random
import argparse
import matplotlib.pyplot as plt
from PIL import Image
from time import time
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as T

from __future__ import print_function
from dbpn import Net as DBPN
from dbpn import get_pair_set
from ssd import SSD
from ssd import build_ssd
from ssd.layers.modules import MultiBoxLoss
from ssd.data.config import voc
from ssd.data import detection_collate
from ssd.data import VOCAnnotationTransform, VOCDetection, BaseTransform, SRDetection, DBPNLoader
from ssd.eval import test_net

import sys; sys.argv=['']; del sys



In [3]:
# Validation Setup - We use VOC07 for validation

HOME = os.path.expanduser("~")
voc_root = os.path.join(HOME, "data/VOCdevkit/")

annopath = os.path.join(voc_root, 'VOC2007', 'Annotations', '%s.xml')
imgpath = os.path.join(voc_root, 'VOC2007', 'JPEGImages', '%s.jpg')
imgsetpath = os.path.join(voc_root, 'VOC2007', 'ImageSets', 'Main', '{:s}.txt')
YEAR = '2007'
devkit_path = voc_root + 'VOC' + YEAR
dataset_mean = (104, 117, 123)

set_type = '07val32-5702'
# set_typs is explained below:
"""
We have 5 validation sets to choose, size of 16, 32, 64, 128, 256 repectively,
Due to the time & computation constraint, we have these 5 downsampled val set.
They are randomly(no replacement) picked from  VOC2007/ImageSets/Main/val.txt.

Baseline mAP on Pre-trained ssd300 is measured as well:       (mAP)
These Detection baseline are measured on frozen SSD300 network using
'weights/ssd300_mAP_77.43_v2.pth', for 75x75 LR images been SR-ed by
VOC12 retrained DBPN baseline w. weight 'VOC12-LR-x4-DBPN-ep100.pth'.

~/data/VOCdevkit/VOC2007/ImageSets/Main/07val16-5648.txt      56.48%
~/data/VOCdevkit/VOC2007/ImageSets/Main/07val32-5702.txt      57.02%
~/data/VOCdevkit/VOC2007/ImageSets/Main/07val64-5826.txt      58.26%
~/data/VOCdevkit/VOC2007/ImageSets/Main/07val128-6019.txt     60.19%
~/data/VOCdevkit/VOC2007/ImageSets/Main/07val256-6030.txt     60.30%

These can be used to evaluate training in progress quantatively.
Again we do not run through whole val set for periodical check out, too slow!
"""

def save_img(img, img_path):
    save_img = img.squeeze().clamp(0, 1).numpy().transpose(1,2,0)
    cv2.imwrite(img_path, cv2.cvtColor(save_img*255, cv2.COLOR_BGR2RGB),  [cv2.IMWRITE_PNG_COMPRESSION, 0])

eval_set = DBPNLoader(root='./dataset')
eval_loader = DataLoader(dataset=eval_set, batch_size=1, num_workers=1, shuffle=False)
#torch.cuda.empty_cache()


In [4]:
# Arguments & settings
parser = argparse.ArgumentParser(description='PyTorch Super Resolution Detection Networks')
parser.add_argument('--upscale_factor', type=int, default=4, help="super resolution upscale factor")
parser.add_argument('--testBatchSize', type=int, default=1, help='testing batch size')
parser.add_argument('--batch_size', type=int, default=4, help='training batch size') # GPU: 9GB!
parser.add_argument('--threads', type=int, default=4, help='number of threads for data loading')
parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')
parser.add_argument('--gpu_mode', type=bool, default=True)
parser.add_argument('--gpus', default=1, type=float, help='number of gpu')
#parser.add_argument('--test_dataset', type=str, default='VOC12-LR-X8-test')
#parser.add_argument('--sr_dataset', type=str, default='VOC12-SR-X8')
parser.add_argument('--train_dataset', type=str, default='VOC12-LR-x4')
parser.add_argument('--hr_dataset', type=str, default='VOC12-HR')
parser.add_argument('--anno_path', type=str, default='Annotations')
parser.add_argument('--imSetpath', type=str, default='ImageSets')
parser.add_argument('--input_dir', type=str, default='./dataset')
#parser.add_argument('--output', default='./dataset/results', help='Location to save some outputs')
parser.add_argument('--checkpt', default='./checkpoint', help='Location to save checkpoint models')
parser.add_argument('--lr', type=float, default=1e-4, help='Learning Rate. Default=0.0001')
parser.add_argument('--nEpochs', type=int, default=5, help='number of epochs to fine tune net S over target loss')

opt = parser.parse_args()

gpus_list=range(opt.gpus)
print(opt)

cuda = opt.gpu_mode
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

torch.manual_seed(opt.seed)
if cuda:
    torch.cuda.manual_seed(opt.seed)
    torch.cuda.empty_cache()


Namespace(anno_path='Annotations', batch_size=4, checkpt='./checkpoint', gpu_mode=True, gpus=1, hr_dataset='VOC12-HR', imSetpath='ImageSets', input_dir='./dataset', lr=0.0001, nEpochs=5, seed=123, testBatchSize=1, threads=4, train_dataset='VOC12-LR-x4', upscale_factor=4)


In [5]:
print('===> Loading net S fine-tune training datasets')
# VOC0712 dataset mean
MEANS = (104, 117, 123)

sd_dataset = SRDetection(root='./dataset', image_sets='trainval.txt', data_mean = MEANS,
                         target_transform = VOCAnnotationTransform())

sd_data_loader = DataLoader(dataset=sd_dataset, batch_size=opt.batch_size, num_workers=opt.threads,
                            shuffle=False, collate_fn=detection_collate, pin_memory=True)


===> Loading net S fine-tune training datasets


In [ ]:
num_classes = 21                                  # +1 for background
ssd300net = build_ssd('test', 300, num_classes)   # initialize SSD
ssd300net.load_state_dict(torch.load('ssd/weights/ssd300_mAP_77.43_v2.pth'))
ssd300net.eval()

print('Finished loading SSD300 evaluation model!')
# load DBPN SRed data, and run evaluation test.
eval_det = VOCDetection(voc_root, [('2007', set_type)],
                       BaseTransform(300, dataset_mean),
                       VOCAnnotationTransform(), sr_path='./dataset/VOC07-SR-x4')
if cuda:
     ssd300net = ssd300net.cuda()
     cudnn.benchmark = True
# evaluation

test_net('./eval', ssd300net, cuda, eval_det,
         BaseTransform(ssd300net.size, dataset_mean), 5, 300, thresh=0.01)


In [ ]:
# TODO:
# 1. come up with test/eval functions
# 2. come up with visualization image

class DBPN2SSD(nn.Module):
    
    def __init__(self, s_model_name, d_model_name, d_frozen):
        super(DBPN2SSD, self).__init__()
        self.supervis = DBPN(num_channels=3, base_filter=64, feat=256, num_stages=7, scale_factor=4)
        if os.path.exists(s_model_name):
            self.supervis = torch.nn.DataParallel(self.supervis, device_ids=gpus_list)
            self.supervis.load_state_dict(torch.load(s_model_name, map_location=lambda storage, loc: storage))

        # self.detector = SSD(), setup ssd as 'train' mode for gradient flow
        # later at test/eval situation, we will overwrite it's mode to 'test'
        self.detector = build_ssd('train', 300, 21)
        if os.path.exists(d_model_name):
            self.detector.load_state_dict(torch.load(d_model_name, map_location=lambda storage, loc: storage))
        if d_frozen:
            for param in self.detector.parameters():
                param.requires_grad = False
        
    def forward(self, x):
        superx = self.supervis(x)
        # current detector: SSD300, so assume superx: 300x300!
        detect = self.detector(superx)
        return (superx, detect)



In [ ]:
SRDnet = DBPN2SSD('dbpn/models/VOC12-LR-x4-DBPN-ep100.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)

#dbpnet = SRDnet.supervis
dbpnet = SRDnet

if cuda:
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    # dbpnet = torch.nn.DataParallel(dbpnet)
    cudnn.benchmark = True
    dbpnet = dbpnet.cuda(gpus_list[0])

#dbpn_eval(dbpnet, eval_set, cuda)

eval_set = DBPNLoader(root='./dataset')
eval_loader = DataLoader(dataset=eval_set, batch_size=1, num_workers=1, shuffle=False)

#def dbpn_eval(dbpnet, eval_loader, cuda):
dbpnet.eval()
#num_images = len(eval_set)
#for i in range(num_images):
for iteration, batch in enumerate(eval_loader, 1):
    # input is LR image; sr_target is pseudo (300x300) original HR image, det_target is bboxes
    # input, sr_target = batch[0], batch[1]
    lr, hr, fname = batch[0], batch[1], batch[2]
    # lr, hr, fname = eval_set.pull_item(i)
    if cuda:
        lr_v = Variable(lr.cuda(gpus_list[0]))
    else:
        lr_v = Variable(lr)
    sr, _ = dbpnet(lr_v)
    srimg = sr.cpu().data
    image=srimg
    image = (image - image.min())/(image.max() - image.min())
    srimg=image
    #save_img = srimg.squeeze().clamp(0, 1).numpy().transpose(1,2,0)
    imagef = os.path.join('./dataset/VOC07-SR-x4/', '%s.jpg')
    save_img(srimg, imagef % fname)
    #save_fn = imagef % fname
    #print(save_fn)
    #print(save_img.shape)
    #cv2.imwrite(save_fn, cv2.cvtColor(save_img * 255, cv2.COLOR_BGR2RGB),  [cv2.IMWRITE_PNG_COMPRESSION, 0])

#dbpnet.train()



In [6]:
num_classes = 21                                  # +1 for background
ssd300net = build_ssd('test', 300, num_classes)   # initialize SSD
ssd300net.load_state_dict(torch.load('ssd/weights/ssd300_mAP_77.43_v2.pth'))
ssd300net.eval()

print('Finished loading SSD300 evaluation model!')
# load DBPN SRed data, and run evaluation test.
eval_det = VOCDetection(voc_root, [('2007', set_type)],
                       BaseTransform(300, dataset_mean),
                       VOCAnnotationTransform(), sr_path='./dataset/VOC07-SR-x4')
if cuda:
     ssd300net = ssd300net.cuda()
     cudnn.benchmark = True
# evaluation

test_net('./eval', ssd300net, cuda, eval_det,
         BaseTransform(ssd300net.size, dataset_mean), 5, 300, thresh=0.01, eval_set=set_type)


/home/x/cs231n/super-resolution-detection/ssd/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/x/cs231n/super-resolution-detection/ssd/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


Finished loading SSD300 evaluation model!
Evaluating detections
VOC07 metric? Yes
Mean AP = 0.5669
~~~~~~~~


In [ ]:
torch.cuda.empty_cache()


In [ ]:
SRDnet = DBPN2SSD('dbpn/models/VOC12-LR-x4-DBPN-ep100.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)

#dbpnet = SRDnet.supervis
dbpnet = SRDnet

if cuda:
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    dbpnet = torch.nn.DataParallel(dbpnet)
    cudnn.benchmark = True
    dbpnet = dbpnet.cuda(gpus_list[0])

dbpn_eval(dbpnet, eval_set, cuda)



In [ ]:
# Hyperparameters - balancing weights of net S loss vs. net D loss
alpha = 1.0
beta = 1.0

decay_steps = [5000, 10000, 20000, 40000, 80000, 160000]
gamma = 0.1
def adjust_learning_rate(optimizer, gamma=0.1, step=0):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = opt.lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def train():
    #SDnet = DBPN2SSD('dbpn/models/DBPN_x4.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
    #SDnet = DBPN2SSD('checkpoint/SRx4_4000.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
    SDnet = DBPN2SSD('dbpn/models/VOC12-LR-x4-DBPN-ep100.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
    
    net = SDnet

    # Criterions:
    #
    # net SR loss function, change to L2 later
    criterion_sr = nn.MSELoss()

    # ssd loss function
    criterion_ssd = MultiBoxLoss(voc['num_classes'], 0.5, True, 0, True, 3, 0.5, False, use_gpu=cuda)

    if cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        #net.supervis = torch.nn.DataParallel(net.supervis)
        cudnn.benchmark = True
        net = net.cuda(gpus_list[0])
        criterion_sr = criterion_sr.cuda(gpus_list[0])
        criterion_ssd = criterion_ssd.cuda(gpus_list[0])


    # optimizer = optim.SGD(net.supervis.parameters(), lr=opt.lr, \
    #                       momentum=opt.momentum, weight_decay=opt.weight_decay)
    #
    # Use universal Adam first :)
    # we can be Specific to what part of network's parameters to optimize, here is SuperR Net
    optimizer = optim.Adam(net.supervis.parameters(), lr=opt.lr, betas=(0.9, 0.999), eps=1e-8)
    epoch_loss = 0
    
    # set model to training mode
    net.train()
    
    epoch = 1
    max_iter = 20000
    decay_step = 0
    for iteration, batch in enumerate(sd_data_loader, 1):
        
        if iteration == max_iter:
            break
        if iteration in decay_steps:
            decay_step += 1
            adjust_learning_rate(optimizer, gamma, decay_step)
            
        # input is LR image; sr_target is pseudo (300x300) original HR image, det_target is bboxes
        input, sr_target, det_target = batch[0], batch[1], batch[2]
        if cuda:
            input = Variable(input.cuda(gpus_list[0]))
            sr_target = Variable(sr_target.cuda(gpus_list[0]))
            det_target = [Variable(ann.cuda(gpus_list[0]), volatile=True) for ann in det_target]
        else:
            input = Variable(input)
            sr_target = Variable(sr_target)
            det_target = [Variable(ann, volatile=True) for ann in det_target]

        optimizer.zero_grad()
        t0 = time()
        
        sr_out,ssd_out = net(input)
        loss_sr = criterion_sr(sr_out, sr_target)
        
        # Compound Loss from net-S: loss_sr and net-D: loss_l, loss_c
        loss_l, loss_c = criterion_ssd(ssd_out, det_target)
        loss = loss_sr * alpha + beta *(loss_l + loss_c)
        
        epoch_loss += loss.data[0]
        loss.backward()
        optimizer.step()
        t1 = time()

        if iteration != 0 and iteration % 2000 == 0:
            print('Saving state, iter:', iteration)
            torch.save(net.supervis.state_dict(), opt.checkpt + '/' + 'SRx4_' + repr(iteration+4000) + '.pth')
        
        print("===> Epoch[{}]({}/{}): Loss: {:.4f} || Timer: {:.4f} sec.".format(epoch, \
                    iteration, len(sd_data_loader), loss.data[0], (t1 - t0)))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(sd_data_loader)))


In [ ]:
train()


In [ ]:
def save_img(img, img_name):
    save_img = img.squeeze().clamp(0, 1).numpy().transpose(1,2,0)

    # save img
    save_dir=os.path.join(opt.output,opt.test_dataset)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    save_fn = save_dir +'/'+ img_name
    cv2.imwrite(save_fn, cv2.cvtColor(save_img*255, cv2.COLOR_BGR2RGB),  [cv2.IMWRITE_PNG_COMPRESSION, 0])

    
def eval():

    net = DBPN2SSD('dbpn/models/VOC12-LR-x4-DBPN-ep100.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
    #net = DBPN2SSD('dbpn/models/DBPN_x4.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)

    if cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        #net.supervis = torch.nn.DataParallel(net.supervis)
        cudnn.benchmark = True
        net = net.cuda(gpus_list[0])

    net.eval()
    
    pred_boxes = []
    pred_labels = []
    pred_scores = []
    gt_boxes = []
    gt_labels = []

    ###img_mean = np.array(MEANS, dtype=np.float32)

    #box_coder = SSDBoxCoder()
    #for iteration, batch in enumerate(eval_data_loader, 1):
    for iteration, batch in enumerate(sd_data_loader, 1):
        
        # input is LR image; sr_target is pseudo (300x300) original HR image, det_target is bboxes
        # input, sr_target = batch[0], batch[1]
        input, sr_target, det_target = batch[0], batch[1], batch[2]

        if cuda:
            input_v = Variable(input.cuda(gpus_list[0]), volatile=True)
            #sr_target = Variable(sr_target.cuda(gpus_list[0]))
            ###ssd_target = [Variable(ann.cuda(gpus_list[0]), volatile=True) for ann in det_target]
        else:
            input_v = Variable(input, volatile=True)
            #sr_target = Variable(sr_target)
            ###ssd_target = [Variable(ann, volatile=True) for ann in det_target]

        sr_img, ssd_out = net(input_v)
        ###sr_img = net(input_v)
        ###sr_img = sr_img.cpu().detach().numpy()
        srimg = sr_img.cpu().data
        image=srimg
        image = (image - image.min())/(image.max() - image.min())
        srimg=image
        save_img(srimg, 'SuperRtest.jpg')
        
        #img_input = input_v.squeeze().cpu().numpy().transpose(1,2,0) + img_mean
        img_input = input_v.squeeze().cpu().numpy().transpose(1,2,0)
        img_input = img_input*255
        img_input = np.clip(img_input, 0, 255)
        #plt.imshow(sr_target.squeeze().clamp(0, 1).numpy().transpose(1,2,0))
        #plt.imshow(sr_target[0].transpose((1,2,0)))
        print(np.max(img_input))
        print(np.min(img_input))

        ### all good so far!!!
        plt.imshow(img_input.astype(np.uint8))
        plt.show()

        
        #img0 = Image.fromarray(sr_img[0].transpose((1,2,0)), 'RGB')
        #plt.imshow(sr_img[0].transpose((1,2,0)))
        #plt.show()

        rimg = srimg.numpy()
        # debug SR imge display issue, clue: sr_img range from DBPN output [-11070.912, 12110.84] !!!
        print(rimg.shape)
        print(rimg[0].shape)
        mx = np.max(rimg)
        mi = np.min(rimg)
        gp = mx - mi
        print(mx)
        print(mi)
        #img_sr = sr_img.squeeze().transpose(1,2,0) + img_mean
        #img_sr = (rimg.squeeze().transpose(1,2,0) /- mi) / gp + img_mean / 255.
        #img_sr = ((rimg.squeeze().transpose(1,2,0) / mx) + 1.0) / 2.0 + img_mean / 255.
        img_sr = rimg.squeeze().transpose(1,2,0)
        #img_sr = img_sr*255 + img_mean
        img_sr = img_sr*255
        img_sr = np.clip(img_sr, 0, 255)
        ###img_sr = img_sr * 255.
        ###img_sr = np.clip(img_sr, 0, 255)
        ###plt.imshow(img_sr.astype(np.uint8))
        ###img_sr = srimg.squeeze().clamp(0, 1).numpy().transpose(1,2,0)
        
        # !! after use eval_data_loader (instead of sd_data_loader) this SRed image show great !!
        plt.imshow(img_sr.astype(np.uint8))
        plt.show()
        ###print(np.max(img_sr))
        ###print(np.min(img_sr))


        
        print(sr_target.shape)
        print(sr_target[0].shape)
        #img1 = Image.fromarray(sr_img[0].transpose((1,2,0)), 'RGB')
        #img1 = sr_target.squeeze().numpy().transpose(1,2,0) + img_mean
        #img_target = sr_target.squeeze().numpy().transpose(1,2,0) + img_mean
        img_target = sr_target.squeeze().numpy().transpose(1,2,0)
        img_target = img_target
        img_target = np.clip(img_target, 0, 255)
        #plt.imshow(sr_target.squeeze().clamp(0, 1).numpy().transpose(1,2,0))
        #plt.imshow(sr_target[0].transpose((1,2,0)))
        print(np.max(img_target))
        print(np.min(img_target))

        ### all good so far!!!
        plt.imshow(img_target.astype(np.uint8))
        plt.show()

        
        target = sr_target.numpy()
        
        # create PSNR/SSIM
        s_img = sr_img
        #ssim = metric_ssim(target,sr_img)
        #psnr = metric_psnr(target,sr_img)
        
        # Get the target box and lables
        #print("ssim, psnr",ssim,psnr)
        ###box_targets = ssd_target[0][:,:-1]
        ###label_targets = ssd_target[0][:,-1]
        ###gt_boxes.append(box_targets.squeeze(0))
        ###gt_labels.append(label_targets.squeeze(0))

        ###loc_preds = ssd_out[0][:,:-1]
        ###cls_preds = ssd_out[0][:,-1]
        #box_preds, label_preds, score_preds = box_coder.decode(
        #    loc_preds.cpu().data.squeeze(),
        #    F.softmax(cls_preds.squeeze(), dim=1).cpu().data,
        #    score_thresh=0.01)

        #pred_boxes.append(box_preds)
        #pred_labels.append(label_preds)
        #pred_scores.append(score_preds)
        ## img = Image.fromarray(sr_img[0].transpose((1,2,0)), 'RGB')
        ## target = Image.fromarray(target[0].transpose((1,2,0)), 'RGB')
        ## img = sr_img.squeeze().numpy().transpose(1,2,0) + img_mean
        
        ### sr_img changed by metric_ssim or metric_psnr functions ???
        #####img = s_img.squeeze().transpose(1,2,0) + img_mean
        #####img = np.clip(img, 0, 255)
        #draw = ImageDraw.Draw(img)
        #for box in loc_preds:BaseTransform
            #print("listbox",list(box))
            #draw.rectangle(list(box), outline='red')
        #print("sr_img shape",sr_target.shape)        
        #####plt.imshow(img.astype(np.uint8))
        #####plt.show()

        #target = sr_target.squeeze().numpy().transpo-se(1,2,0) + img_mean
        
        ## why here shows the reversed (negative) image, 20 lines above plt.show is good!
        #im_target = sr_target.squeeze().numpy().transpose(1,2,0) + img_mean
        im_target = sr_target.squeeze().numpy().transpose(1,2,0)
        im_target = im_target
        im_target = np.clip(im_target, 0, 255)
        

        plt.imshow(im_target.astype(np.uint8))
        plt.show()
        
        save_img(torch.from_numpy(sr_target),"test.jpeg")

        break


In [ ]:
torch.cuda.empty_cache()


### Debugging code below, no need to run :)

In [ ]:
print(' YOU Donot need to run this ===> Loading some datasets')

lr_path = os.path.join(opt.input_dir, opt.train_dataset)
hr_path = os.path.join(opt.input_dir, opt.hr_dataset)

fine_train_set = get_pair_set(lr_path, hr_path)
train_data_loader = DataLoader(dataset=fine_train_set, num_workers=opt.threads, \
                               batch_size=opt.testBatchSize, shuffle=False)


In [ ]:
def simple_test():
    SDnet = DBPN2SSD('dbpn/models/DBPN_x4.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
    net = SDnet
    
    if cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True
        net = net.cuda()

    else:
        torch.set_default_tensor_type('torch.FloatTensor')


    
simple_test()

In [ ]:
# dtype = torch.FloatTensor
dtype = torch.cuda.FloatTensor ## UNCOMMENT THIS LINE IF YOU'RE ON A GPU!

# Real samples were all associated with Ture labels - 1
T_labels = torch.ones(logits_real.size()).type(dtype)
# Fake samples were all associated with Fake labels - 0
F_labels = torch.zeros(logits_fake.size()).type(dtype)
